# **Lab 2: Matrix Factorization**
**Kevin Arnmark**

# **Abstract**

In this lab I implement algorithms for solving sparse matrix-vector product, QR factorization, solving Ax=b and blocked matrix-matrix product. The base algorithms and theory comes from the lecture notes. The accuracy of the algorithms are also tested in various ways.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [ ]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [1]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common 
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *
    
#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

The following functions are implemented in this report:

1. **Function:** sparse matrix-vector product

  **Input:** vector $x$, sparse (real, quadratic) matrix $A$: CRS arrays val, col_idx, row_ptr
  
  **Output:** matrix-vector product $b=Ax$
  
  **Test:** verify accuracy against dense matrix-vector product. 

2. **Function:** QR factorization

  **Input:** (real quadratic) matrix $A$
  
  **Output:** orthogonal matrix $Q$, upper triangular matrix $R$, such that $A=QR$
  
  **Test:** $R$ upper triangular, Frobenius norms $|| Q^TQ-I ||_F, || QR-A ||_F$

3. **Function:** direct solver $Ax=b$

  **Input:** (real, quadratic) matrix $A$, vector $b$
  **Output:** vector x=A^-1b
  **Test:** residual || Ax-b ||, and || x-y || where y is a manufactured solution with b=Ay

**Extra Task:**

4. **Function:** blocked matrix-matrix product

  **Input:** (real compatible) matrices $A, B$
  
  **Output:** matrix-matrix product $C=AB$
  
  **Test:** formulate test cases to verify accuracy. 


# **Method**

**Sparse matrix-vector product**

The compressed row storage format (CRS) takes the form of three arrays. One array containing the non zero values, another containing those values column indices, and the last pointers to the start of each row. I created a new class for this to keep the algorithm structured and easy to follow. (Chapter 5.8)

The algorithm used below is from Chapter 5.8 Algorithm 5.9. The changes I made compared to the lecture notes are that I use 0 indexing instead of 1 indexing.

In [2]:
class CRS:
  #val, col_idx, row_ptr = [],[],[]
  def __init__(self, val, col_idx, row_ptr):
    assert len(val) == row_ptr[-1]
    self.val = val
    self.col_idx = col_idx
    self.row_ptr = row_ptr


def crs_matrix_vector_product(x, A):
  b = np.zeros((len(x), ))
  for i in range(len(x)):
    for j in range(A.row_ptr[i], A.row_ptr[i + 1]):
      b[i] += A.val[j] * x[A.col_idx[j]]
  return b

**QR Factorization**

QR factorization is a way to factorize a matrix $A$ by creating an orthogonal matrix $Q$ and an upper triangular matrix $R$ where $QR=A$. (Chapter 5.3)

I am using the modified gram-schmidt factorization Algorithm 5.3 from Chapter 5.3. It is also possible to use the householder factorization for solving this.

In [3]:
def qr_factorization(A):
  n = len(A)
  R, Q = np.zeros((n, n)), np.zeros((n, n))
  for j in range(len(A)):
    v = np.array(A[:,j])
    for i in range(j):
      R[i,j] = np.dot(Q[:,i], v)
      v = v - R[i,j]*Q[:,i]
    R[j, j] = np.linalg.norm(v)
    Q[:,j] = v[:] / R[j,j]

  return Q, R

**Direct Solver Ax=b**

Solving $Ax=B$ is done by inverting matrix $A$ resulting in $x=A^{-1}b$. By factorizing $A$ into an orthogonal matrix $Q$ and upper triangular matrix $R$ it is easier to get the inverse of $A$. $x=A^{-1}b \iff x=(QR)^{-1}b$.

The inverse of $Q$ is the transpose of $Q$ since it is an orthogonal matrix. $Q^{-1} = Q^T$ 

The inverse of an upper triangular matrix can be calculated using backward substitution. The algorithm below implements an algorithm for solving backward substitution from Chapter 5.2 Algorithm 5.2.

In [4]:
def backward_substitution(U, b):
  n = len(b)
  x = np.zeros((n, ))
  x[n - 1] = b[n - 1] / U[n-1, n-1]
  for i in range(n - 2, -1, -1):
    s = 0
    for j in range(i + 1, n):
      s += U[i, j] * x[j]
    x[i] = (b[i] - s) / U[i, i]
  return x

def direct_solver(A, b):
  assert len(A[0]) == len(b)
  Q, R = qr_factorization(A)
  Q_i = np.transpose(Q)
  return backward_substitution(R, Q_i.dot(b))

**Extra task Blocked matrix-matrix product**

By dividing up each matrix into smaller blocks it is possible to reduce the number of memory references needed to calculate a matrix-matrix product. 

The number of memory references needed with a blocked matrix-matrix product is $m=n^2(2N+1)$ instead of $m=2n^2+n^3$, which is the case for Algorithm 5.7, where $N$ is the number of blocks and $n$ the dimension of each matrix. (Chapter 5.7)

The algorithm is based on the algorithm from Chapter 5.7 Algorithm 5.8. In this algorithm i am creating as large blocks as possible since I am not going to use any matrices that divided are larger than the cache. By using as large blocks as possible the computational intensity is maximized.

In [5]:
def blocked_m_m_product(A, B):
  assert len(A[0]) == len(B)
  M, N, P = 2, 2, 2
  m, n, p = len(A), len(B[0]), len(B)
  bm, bn, bp = m/M, n/N, p/P
  while (not bm.is_integer()): # Choosing as large blocks as possible
    bm = m/M
    M += 1
  while (not bn.is_integer()):
    bn = n/N
    N += 1
  while (not bp.is_integer()):
    bp = p/P
    P += 1
  bm = int(bm)
  bn = int(bn)
  bp = int(bp)
  C = np.zeros((m, n))
  for i in range(M):
    for j in range(N):
      for k in range(P):
        C[i*bm:(i+1)*bm, j*bn:(j+1)*bn] += np.matmul(A[i*bm:(i+1)*bm, k*bp:(k+1)*bp], B[k*bp:(k+1)*bp, j*bn:(j+1)*bn])

  return C

# Tests:

Testing the accuracy of the algorithms. This could be done several different ways. I did a smaller amount of predetermined tests, but ultimately I would have wanted to implement some random generation of matrices and vectors to be able to run more tests to get a higher significance of the tests. But I unfortunately did not have the time for that.

In [6]:
def frobenius_norm(A):
  s = 0
  for i in range(len(A)):
    for j in range(len(A[0])):
      s += abs(A[i,j]**2)
  return np.sqrt(s)

# Testing sparse matrix-vector product:

A = np.array([[0, 0, -2],[0, 1, 0],[9, 0, 0]])
v = np.array([0, 3, 2])
A_crs = CRS(np.array([-2, 1, 9]), np.array([2, 1, 0]), np.array([0, 1, 2, 3]))


# Test: verify accuracy against dense matrix-vector product. 
assert (crs_matrix_vector_product(v, A_crs) - np.dot(A, v)).all() == 0
print("sparse matrix-vector product tests passed")

# Testing qr_factorization(A)

A = np.array([[1, 3, 2],[2, 1, 3],[3, 2, 1]])
Q, R = qr_factorization(A)


#Test: R upper triangular
assert np.allclose(R, np.triu(R)) == True
#Test: Frobenius norm || Q^TQ-I ||_F
np.testing.assert_almost_equal(frobenius_norm(np.matmul(np.transpose(Q), Q) - np.identity(len(Q))), 0)
#Test: Frobenius norm || QR-A ||_F
np.testing.assert_almost_equal(frobenius_norm(np.matmul(Q, R) - A), 0)

print("qr factorization tests passed")

# Testing direct solver Ax = b
b = np.array([1, 2, 3])
x = direct_solver(A, b)
# Test: residual || Ax-b ||
np.testing.assert_almost_equal(np.linalg.norm(np.dot(A,x) - b), 0)

y = np.array([5, -3, 1])
b = np.dot(A, y)
x = direct_solver(A, b)
# Test: || x-y || where y is a manufactured solution with b=Ay
np.testing.assert_almost_equal(np.linalg.norm(x - y), 0)

print("direct solver Ax = b tests passed")

# Testing the blocked matrix-matrix product

A = np.array([[1, 2, 3, 4],[4, 3, 2, 1],[1, 3, 2, 4],[4, 2, 3, 1]])
B = np.array([[1, -2, 2, 4],[3, 3, 4, 0],[1, 2, 1, -4],[-4, 1, 2, 0]])

# Test: testing against numpy matmul to verify accuracy
assert (np.matmul(A, B) - blocked_m_m_product(A, B)).all() == 0

A = np.array([[1, 2, 3],[4, 3, 2],[1, 3, 2],[4, 2, 3]])
B = np.array([[1, -2], [3, 3], [1, 2]])
# Test: testing against numpy matmul to verify accuracy.
assert (np.matmul(A, B) - blocked_m_m_product(A, B)).all() == 0
print("blocked matrix-matrix product tests passed")

sparse matrix-vector product tests passed
qr factorization tests passed
direct solver Ax = b tests passed
blocked matrix-matrix product tests passed


# **Results**

The algorithms passes all the tests, meaning that they are of sufficient accuracy. They do however have limited precision.

# **Discussion**

If I had more time I would have wanted to write some test to verify that the blocked matrix-matrix product uses less memory references. It is currently only testing the accuracy of the product.